https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/
    
    

In [51]:
import numpy as np

In [52]:
from nltk import word_tokenize, tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer

from keras.preprocessing.text import Tokenizer, one_hot
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Flatten, Dropout, LSTM
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from gensim.models import Word2Vec

import re, time
import string

from sklearn.decomposition import PCA
from matplotlib import pyplot as plt

%matplotlib inline




In [53]:
#filename = 'CompleteShakespeare.txt'
filename = 'hamlet2.txt'
fid  = open(filename)
raw_text = fid.read()
fid.close()

In [54]:
len(raw_text)


177659

In [55]:
#chars = sorted(list(set(raw_text)))
#print(len(chars))
#print(chars)
print( raw_text)


THE TRAGEDY OF HAMLET, PRINCE OF DENMARK




by William Shakespeare






Contents

ACT I Scene I. Elsinore. A platform before the Castle. Scene II.
Elsinore. A room of state in the Castle Scene III. A room in Polonius’s
house. Scene IV. The platform. Scene V. A more remote part of the
Castle.

ACT II Scene I. A room in Polonius’s house. Scene II. A room in the
Castle.

ACT III Scene I. A room in the Castle. Scene II. A hall in the Castle.
Scene III. A room in the Castle. Scene IV. Another room in the Castle.

ACT IV Scene I. A room in the Castle. Scene II. Another room in the
Castle. Scene III. Another room in the Castle. Scene IV. A plain in
Denmark. Scene V. Elsinore. A room in the Castle. Scene VI. Another
room in the Castle. Scene VII. Another room in the Castle.

ACT V Scene I. A churchyard. Scene II. A hall in the Castle.




 Dramatis Personæ

HAMLET, Prince of Denmark. CLAUDIUS, King of Denmark, Hamlet’s uncle.
The GHOST of the late king, Hamlet’s father. GERTRUDE, the Queen,


In [56]:
clean_text = raw_text.lower()

In [57]:
chars = sorted(list(set(clean_text)))
print(len(chars))
print(chars)


44
['\n', ' ', '!', '&', ',', '-', '.', ':', ';', '?', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'à', 'æ', '—', '‘', '’']


In [58]:
print(raw_text[10000:11000])
print(clean_text[10000:11000])

with wisest sorrow think on him,
Together with remembrance of ourselves. Therefore our sometime sister,
now our queen, Th’imperial jointress to this warlike state, Have we, as
’twere with a defeated joy, With one auspicious and one dropping eye,
With mirth in funeral, and with dirge in marriage, In equal scale
weighing delight and dole, Taken to wife; nor have we herein barr’d
Your better wisdoms, which have freely gone With this affair along. For
all, our thanks. Now follows, that you know young Fortinbras, Holding a
weak supposal of our worth, Or thinking by our late dear brother’s
death Our state to be disjoint and out of frame, Colleagued with this
dream of his advantage, He hath not fail’d to pester us with message,
Importing the surrender of those lands Lost by his father, with all
bonds of law, To our most valiant brother. So much for him. Now for
ourself and for this time of meeting: Thus much the business is: we
have here writ To Norway, uncle of young Fortinbras, Who, impoten

In [59]:
print( re.sub(' +', ' ',clean_text))

the tragedy of hamlet, prince of denmark




by william shakespeare






contents

act i scene i. elsinore. a platform before the castle. scene ii.
elsinore. a room of state in the castle scene iii. a room in polonius’s
house. scene iv. the platform. scene v. a more remote part of the
castle.

act ii scene i. a room in polonius’s house. scene ii. a room in the
castle.

act iii scene i. a room in the castle. scene ii. a hall in the castle.
scene iii. a room in the castle. scene iv. another room in the castle.

act iv scene i. a room in the castle. scene ii. another room in the
castle. scene iii. another room in the castle. scene iv. a plain in
denmark. scene v. elsinore. a room in the castle. scene vi. another
room in the castle. scene vii. another room in the castle.

act v scene i. a churchyard. scene ii. a hall in the castle.




 dramatis personæ

hamlet, prince of denmark. claudius, king of denmark, hamlet’s uncle.
the ghost of the late king, hamlet’s father. gertrude, the queen,


In [60]:
clean_text = clean_text.replace('\t',' ')
clean_text = clean_text.replace('\n',' ')
clean_text = clean_text.replace('!','')
clean_text = clean_text.replace('&','')
clean_text = clean_text.replace(',','')
clean_text = clean_text.replace('-','')
clean_text = clean_text.replace(':','')
clean_text = clean_text.replace(';','')
clean_text = clean_text.replace('?','')
clean_text = clean_text.replace('[', '')
clean_text = clean_text.replace(']', '')
clean_text = clean_text.replace('_', '')
clean_text = clean_text.replace('—', '')
clean_text = clean_text.replace('‘', '')
clean_text = clean_text.replace('’', '')

chars = sorted(list(set(clean_text)))
numUniqueChars = len(chars)
print(chars)
char_to_int = dict((c, i) for i, c in enumerate(chars))
#char_to_int

#string.punctuation
#tokens = [punctuation.sub('', w) for w in tokens]

[' ', '.', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'à', 'æ']


In [61]:
# This is the exact reverse of the dictionary defined above.  We want to be able to convert this back to text

int_to_char = dict((i, c) for i, c in enumerate(chars))

In [62]:
numChars = len(clean_text)

In [63]:
sequence_length = 100

x_train = []
y_train = []
for i in range(0,numChars-sequence_length-1):
    sequence_x = clean_text[i:i+sequence_length]
    sequence_y = clean_text[i+sequence_length]
    
    x_numeric = [char_to_int[x] for x in sequence_x]
    y_numeric = char_to_int[sequence_y]

    x_train.append(x_numeric)
    y_train.append(y_numeric)


In [64]:
numElems = len(x_train)
print(numElems)


171106


In [65]:
x_train_lstm = np.reshape(x_train, (numElems, sequence_length, 1))

In [66]:
# Normalize x_train

x_train_lstm = x_train_lstm / float(numUniqueChars)  

In [67]:
y_train = np_utils.to_categorical(y_train)

In [68]:
y_train.shape

(171106, 30)

In [69]:
y_train[2500]

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

# Define and train an LSTM in Keras

In [70]:
model = Sequential()
model.add(LSTM(256, input_shape=(x_train_lstm.shape[1], x_train_lstm.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer = 'adam')

In [71]:
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]



In [72]:
t = time.time()
num_epochs = 50

model.fit(x_train_lstm, y_train, epochs=num_epochs, batch_size=64, callbacks=callbacks_list)

elapsed = time.time()-t
print('Elapsed Time: ' + str(elapsed))

Epoch 1/5
  4992/171106 [..............................] - ETA: 18:01 - loss: 2.9982

KeyboardInterrupt: 

In [ ]:
#model.load_weights('./weights-improvement-01-2.6285.hdf5')
#t = time.time()
#
#model.fit(x_train_lstm, y_train, epochs=num_epochs, batch_size=64, callbacks=callbacks_list)
#
#elapsed = time.time()-t
#print('Elapsed Time: ' + str(elapsed))

# Now how do we generate text?

Using the weight generated file above

In [65]:
#weightfile = 'weights-improvement-25-1.6520.hdf5'#'weights-improvement-20-2.0764.hdf5' 
weightfile = 'weights-improvement-05-1.8148.hdf5'#'weights-improvement-01-2.6285.hdf5'
model.load_weights(weightfile)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [67]:
starting_point = x_train[np.random.randint(0, len(x_train)-1)]
x = np.reshape(starting_point, (1, len(starting_point), 1))
x = x / float(numUniqueChars)
x

array([[[ 0.2       ],
        [ 0.43333333],
        [ 0.33333333],
        [ 0.3       ],
        [ 0.53333333],
        [ 0.53333333],
        [ 0.16666667],
        [ 0.        ],
        [ 0.7       ],
        [ 0.53333333],
        [ 0.        ],
        [ 0.43333333],
        [ 0.2       ],
        [ 0.06666667],
        [ 0.16666667],
        [ 0.        ],
        [ 0.33333333],
        [ 0.7       ],
        [ 0.        ],
        [ 0.06666667],
        [ 0.66666667],
        [ 0.        ],
        [ 0.7       ],
        [ 0.3       ],
        [ 0.73333333],
        [ 0.66666667],
        [ 0.03333333],
        [ 0.        ],
        [ 0.06666667],
        [ 0.43333333],
        [ 0.2       ],
        [ 0.83333333],
        [ 0.06666667],
        [ 0.5       ],
        [ 0.16666667],
        [ 0.2       ],
        [ 0.63333333],
        [ 0.        ],
        [ 0.16666667],
        [ 0.33333333],
        [ 0.2       ],
        [ 0.16666667],
        [ 0.        ],
        [ 0

In [68]:
# pick a random seed
#start = numpy.random.randint(0, len(dataX)-1)
import sys 

starting_point = x_train[np.random.randint(0, len(x_train)-1)]
for i in range(1000):
    x = np.reshape(starting_point, (1, len(starting_point), 1))
    x = x / float(numUniqueChars)
    
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
#    sequence_initial =  [int_to_char[value] for value in starting_point]
    #[int_to_char[value] for value in (starting_point * numUniqueChars).astype(int)]
    sys.stdout.write(result)
    starting_point.append(index)
    starting_point = starting_point[1:]
#    #output = starting_point[1:len(starting_point)]

    

oe and the sueen to the fard of the sueen to the fard of the sueen to the fard of the sueen to the fall in the father and the sueen to the fard of the sueen to the fard of the sueen to the fard of the sueen to the fall in the father and the sueen to the fard of the sueen to the fard of the sueen to the fard of the sueen to the fall in the father and the sueen to the fard of the sueen to the fard of the sueen to the fard of the sueen to the fall in the father and the sueen to the fard of the sueen to the fard of the sueen to the fard of the sueen to the fall in the father and the sueen to the fard of the sueen to the fard of the sueen to the fard of the sueen to the fall in the father and the sueen to the fard of the sueen to the fard of the sueen to the fard of the sueen to the fall in the father and the sueen to the fard of the sueen to the fard of the sueen to the fard of the sueen to the fall in the father and the sueen to the fard of the sueen to the fard of the sueen to the fard o

[21,
 9,
 6,
 0,
 7,
 2,
 13,
 13,
 0,
 10,
 15,
 0,
 21,
 9,
 6,
 0,
 7,
 2,
 21,
 9,
 6,
 19,
 0,
 2,
 15,
 5,
 0,
 21,
 9,
 6,
 0,
 20,
 22,
 6,
 6,
 15,
 0,
 21,
 16,
 0,
 21,
 9,
 6,
 0,
 7,
 2,
 19,
 5,
 0,
 16,
 7,
 0,
 21,
 9,
 6,
 0,
 20,
 22,
 6,
 6,
 15,
 0,
 21,
 16,
 0,
 21,
 9,
 6,
 0,
 7,
 2,
 19,
 5,
 0,
 16,
 7,
 0,
 21,
 9,
 6,
 0,
 20,
 22,
 6,
 6,
 15,
 0,
 21,
 16,
 0,
 21,
 9,
 6,
 0,
 7,
 2,
 19,
 5,
 0,
 16]

In [70]:
#np.argmax(prediction)
np.random.choice(chars,p=prediction[0])

't'

In [71]:
output

NameError: name 'output' is not defined

In [ ]:
prediction[0]